# ДЗ 8


## Подпружникова\Кулешова Анастасия

Посчитайте Retention по когортам даты установки.
Размер когорты — ___неделя (!)___.

Вам будет дано 2 файла:
1. С событиями установок приложения пользователями (install).
2. С событиями входа в игру (session_start).

### Что такое Retention?

Метрика удержания пользователя. Показывает, какой процент пользователей возвращается в N день
жизни. День жизни — количество дней после установки приложения для пользователя.
В нашем случае мы хотим посчитать Simple Calendar Retention.

___Например:___
В нашей недельной когорте с 2021-05-10 по 2021-05-16 в приложение пришло 200 пользователей.
Это наши 100% пользователей, у которых удержание в день 0 — 100%. На следующий день из этих 100% зашли только 100 человек. Это значит, что удержание на 1-й день
их жизни составило 50%.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
installs = pd.read_csv('les8_installs.csv', index_col = 0).sort_values(by = 'install_date')
logins = pd.read_csv('les8_logins.csv', index_col = 0)

display(installs)
display(logins)

,client_id,install_date
3037,2739543114822951135,2021-01-01
2944,3490824704637873771,2021-01-01
2943,6916724122858110497,2021-01-01
2942,3129921274471558610,2021-01-01
2941,926550100644139460,2021-01-01
...,...,...
789,1362127979312082210,2021-01-30
788,8416153722973627520,2021-01-30
787,1362978427035410694,2021-01-30
793,5951854166351082017,2021-01-30


,client_id,event_timestamp,is_tester,is_youtuber
12,868610407601689670,2021-02-16 07:53:04,0,0
16,2434015632087848193,2021-02-16 07:51:23,0,0
45,2531632462363212905,2021-02-16 07:52:12,0,0
119,3205304356447992721,2021-02-16 07:54:17,0,0
122,7482763148151863115,2021-02-16 07:54:36,0,0
...,...,...,...,...
5844758,374127069099732578,2021-02-15 12:56:25,0,0
5844759,4905473983468376225,2021-02-15 12:57:54,0,0
5844792,110174818069209226,2021-02-15 12:58:05,0,0
5844800,590901015821838791,2021-02-15 12:58:33,0,0


разбить весь месяц на 4 когорты
для этого надо взять первую дату и прибавить к ней 6-7 дней, чтоб получилась неделя
поситать количество установивших игру в первый день недели
далее рассчитать количество уникальных пользователей в последующие дни
финальный столбец это деление количество уникальных пользователей в каждый последующий день
на колиетсво пользователей в первый день недели

## переводим в правильный формат даты и формируем неделю

In [3]:
installs['install_date'] = pd.to_datetime(installs['install_date'], format = '%Y-%m-%d')

installs['week_start'] = installs['install_date'] - installs['install_date'].dt.weekday.astype('timedelta64[D]')

In [4]:
logins['event_timestamp'] = pd.to_datetime(logins['event_timestamp'], format = '%Y-%m-%d %H:%M:%S')

## проверяем на наличие дубликатов и Nan\None

In [5]:
check_dubl_installs = installs.duplicated().sum()
check_nan_installs = installs.isnull().sum()

display(check_dubl_installs) 
display(check_nan_installs) 

0

client_id       0
install_date    0
week_start      0
dtype: int64

In [6]:
check_dubl_logins = logins.duplicated().sum() 
check_nan_logins = logins.isnull().sum()

display(check_dubl_logins) 
display(check_nan_logins) 

4

client_id          0
event_timestamp    0
is_tester          0
is_youtuber        0
dtype: int64

In [7]:
logins = logins.drop_duplicates()

## event_timestamp отбросим часы\минуты\секунды

In [8]:
logins['event_timestamp'] = logins['event_timestamp'].dt.to_period('D').dt.start_time

logins

,client_id,event_timestamp,is_tester,is_youtuber
12,868610407601689670,2021-02-16,0,0
16,2434015632087848193,2021-02-16,0,0
45,2531632462363212905,2021-02-16,0,0
119,3205304356447992721,2021-02-16,0,0
122,7482763148151863115,2021-02-16,0,0
...,...,...,...,...
5844758,374127069099732578,2021-02-15,0,0
5844759,4905473983468376225,2021-02-15,0,0
5844792,110174818069209226,2021-02-15,0,0
5844800,590901015821838791,2021-02-15,0,0


## соеденим таблицы для визульной наглядности

In [9]:
summary_table_installs_logins = installs.merge(
    logins,
    on='client_id',
    how='inner'
)
summary_table_installs_logins

,client_id,install_date,week_start,event_timestamp,is_tester,is_youtuber
0,2739543114822951135,2021-01-01,2020-12-28,2021-01-01,0,0
1,2739543114822951135,2021-01-01,2020-12-28,2021-01-02,0,0
2,2739543114822951135,2021-01-01,2020-12-28,2021-01-02,0,0
3,2739543114822951135,2021-01-01,2020-12-28,2021-01-02,0,0
4,2739543114822951135,2021-01-01,2020-12-28,2021-01-26,0,0
...,...,...,...,...,...,...
286169,1362978427035410694,2021-01-30,2021-01-25,2021-02-04,0,0
286170,1362978427035410694,2021-01-30,2021-01-25,2021-02-04,0,0
286171,1362978427035410694,2021-01-30,2021-01-25,2021-02-04,0,0
286172,5951854166351082017,2021-01-30,2021-01-25,2021-01-30,0,0


In [10]:
summary_table_installs_logins['lifetime'] = (summary_table_installs_logins['event_timestamp'] -
summary_table_installs_logins['install_date'])
summary_table_installs_logins['lifetime'] = summary_table_installs_logins['lifetime'] / np.timedelta64(1, 'D')
summary_table_installs_logins['lifetime'] = summary_table_installs_logins['lifetime'].astype('int')
summary_table_installs_logins

,client_id,install_date,week_start,event_timestamp,is_tester,is_youtuber,lifetime
0,2739543114822951135,2021-01-01,2020-12-28,2021-01-01,0,0,0
1,2739543114822951135,2021-01-01,2020-12-28,2021-01-02,0,0,1
2,2739543114822951135,2021-01-01,2020-12-28,2021-01-02,0,0,1
3,2739543114822951135,2021-01-01,2020-12-28,2021-01-02,0,0,1
4,2739543114822951135,2021-01-01,2020-12-28,2021-01-26,0,0,25
...,...,...,...,...,...,...,...
286169,1362978427035410694,2021-01-30,2021-01-25,2021-02-04,0,0,5
286170,1362978427035410694,2021-01-30,2021-01-25,2021-02-04,0,0,5
286171,1362978427035410694,2021-01-30,2021-01-25,2021-02-04,0,0,5
286172,5951854166351082017,2021-01-30,2021-01-25,2021-01-30,0,0,0


In [11]:
summary_table_installs_logins = summary_table_installs_logins.drop_duplicates()
summary_table_installs_logins

,client_id,install_date,week_start,event_timestamp,is_tester,is_youtuber,lifetime
0,2739543114822951135,2021-01-01,2020-12-28,2021-01-01,0,0,0
1,2739543114822951135,2021-01-01,2020-12-28,2021-01-02,0,0,1
4,2739543114822951135,2021-01-01,2020-12-28,2021-01-26,0,0,25
5,3490824704637873771,2021-01-01,2020-12-28,2021-01-01,0,0,0
6,6916724122858110497,2021-01-01,2020-12-28,2021-01-01,0,0,0
...,...,...,...,...,...,...,...
286165,1362978427035410694,2021-01-30,2021-01-25,2021-02-01,0,0,2
286166,1362978427035410694,2021-01-30,2021-01-25,2021-02-02,0,0,3
286167,1362978427035410694,2021-01-30,2021-01-25,2021-02-04,0,0,5
286172,5951854166351082017,2021-01-30,2021-01-25,2021-01-30,0,0,0


In [12]:
summary_table_installs_logins_nunique = (summary_table_installs_logins
                                         .groupby(['week_start','lifetime'], as_index = False)
                                         .agg({'client_id':'nunique'}))

summary_table_installs_logins_nunique

,week_start,lifetime,client_id
0,2020-12-28,0,2835
1,2020-12-28,1,936
2,2020-12-28,2,592
3,2020-12-28,3,470
4,2020-12-28,4,406
...,...,...,...
622,2021-01-25,107,27
623,2021-01-25,108,18
624,2021-01-25,109,21
625,2021-01-25,110,13


In [13]:
cohort = (summary_table_installs_logins
          .groupby(['week_start'], as_index = False)
          .agg({'client_id':'nunique'})
          .rename(columns = {'client_id':'size_cohort'}))
cohort

,week_start,size_cohort
0,2020-12-28,2835
1,2021-01-04,7382
2,2021-01-11,7013
3,2021-01-18,6771
4,2021-01-25,4423


In [14]:
retenshion = summary_table_installs_logins_nunique.merge(
    cohort,
    on = 'week_start',
    how = 'inner',  
)

retenshion

,week_start,lifetime,client_id,size_cohort
0,2020-12-28,0,2835,2835
1,2020-12-28,1,936,2835
2,2020-12-28,2,592,2835
3,2020-12-28,3,470,2835
4,2020-12-28,4,406,2835
...,...,...,...,...
622,2021-01-25,107,27,4423
623,2021-01-25,108,18,4423
624,2021-01-25,109,21,4423
625,2021-01-25,110,13,4423


In [17]:
retenshion['retenshion'] = retenshion['client_id'] / retenshion['size_cohort']
retenshion.sort_values(by='lifetime').reset_index(drop=True).rename(columns = {'client_id':'count_users'}).head(20)

,week_start,lifetime,count_users,size_cohort,retenshion
0,2021-01-25,-3,1,4423,0.000226
1,2020-12-28,0,2835,2835,1.000000
2,2021-01-04,0,7382,7382,1.000000
3,2021-01-18,0,6771,6771,1.000000
4,2021-01-25,0,4416,4423,0.998417
5,2021-01-11,0,7013,7013,1.000000
6,2021-01-11,1,1924,7013,0.274348
7,2021-01-04,1,2275,7382,0.308182
8,2021-01-25,1,1194,4423,0.269953
9,2020-12-28,1,936,2835,0.330159


In [16]:
# ошибка, которую надо передать разрабам
# также ранеее мы удалили дубликаты этой ошибки, когда чистили вообще дуликаты в смерджиной таблице
summary_table_installs_logins[summary_table_installs_logins['lifetime'] < 0]

,client_id,install_date,week_start,event_timestamp,is_tester,is_youtuber,lifetime
267013,907720504004394920,2021-01-28,2021-01-25,2021-01-25,1,0,-3
